In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [11]:
#path for the file
file_path="pizza.txt"

#reading the file
with open (file_path,"r") as file:
    text=file.read()

    #spliting the data input according to the regex provided
    data=[sentence.strip() for sentence in re.split(r'(?<=[.!?]\s+)',text) if sentence.strip()]



In [12]:
#making an instance of the tokenizer
# tokenizer is used to map each unique word into uniqye integer
tokenizer=Tokenizer()

#scans through all the texts and keeps track of the frequency of each word in the texts
tokenizer.fit_on_texts(data)
# +1 is to act as a place holder for out-of-vocabulary
total_words=len(tokenizer.word_index)+1

In [ ]:
input_sequence=[]
for line in text:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence.append(n_gram_sequence)